In [33]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


In [34]:
cd My Drive

/gdrive/My Drive


In [35]:
cd lightDSFD

/gdrive/My Drive/lightDSFD


In [37]:
cd DSFDv2_r18/

/gdrive/My Drive/lightDSFD/DSFDv2_r18


In [0]:
# !pip install opencv-python

In [45]:
import cv2
print(cv2.__version__)

4.1.2


In [0]:
# 31M44S_1583812904.mp4
#28M43S_1583812723.mp4
#27M43S_1583812663.mp4

In [48]:
import cv2
import os
videoPath='/gdrive/.shortcut-targets-by-id/1rXnNC9eQhwbROXS105Kc4KnEqkvLEi2j/2hours/2020031009/31M44S_1583812904.mp4'
savePath='/gdrive/My Drive/lightDSFD/DSFDv2_r18/videoFrames'
count = 1
# for f in os.listdir(videoPath):
full_path=videoPath
print(full_path)
cap= cv2.VideoCapture(full_path)
i=0
f="31M44S_1583812904.mp4"
%time
while(1):
    ret, frame = cap.read()
    # print(ret)
    # print(frame)
    if ret == True:
      print('Read %d frame: ' % count, ret)
      # save frame as JPEG file
      cv2.imwrite(os.path.join(savePath, "{:s}_{:d}.jpg".format(f,count)), frame)
      count += 1
    else:
      break

%time
  
  


/gdrive/.shortcut-targets-by-id/1rXnNC9eQhwbROXS105Kc4KnEqkvLEi2j/2hours/2020031009/31M44S_1583812904.mp4
CPU times: user 1e+03 ns, sys: 0 ns, total: 1e+03 ns
Wall time: 3.81 µs
Read 1 frame:  True
Read 2 frame:  True
Read 3 frame:  True
Read 4 frame:  True
Read 5 frame:  True
Read 6 frame:  True
Read 7 frame:  True
Read 8 frame:  True
Read 9 frame:  True
Read 10 frame:  True
Read 11 frame:  True
Read 12 frame:  True
Read 13 frame:  True
Read 14 frame:  True
Read 15 frame:  True
Read 16 frame:  True
Read 17 frame:  True
Read 18 frame:  True
Read 19 frame:  True
Read 20 frame:  True
Read 21 frame:  True
Read 22 frame:  True
Read 23 frame:  True
Read 24 frame:  True
Read 25 frame:  True
Read 26 frame:  True
Read 27 frame:  True
Read 28 frame:  True
Read 29 frame:  True
Read 30 frame:  True
Read 31 frame:  True
Read 32 frame:  True
Read 33 frame:  True
Read 34 frame:  True
Read 35 frame:  True
Read 36 frame:  True
Read 37 frame:  True
Read 38 frame:  True
Read 39 frame:  True
Read 40 fram

In [49]:
pwd

'/gdrive/My Drive/lightDSFD/DSFDv2_r18'

In [51]:
cd videoFrames

/gdrive/My Drive/lightDSFD/DSFDv2_r18/videoFrames


In [52]:
! ls -1 | wc -l

3412


In [53]:
!pip install humanfriendly

In [54]:
cd ..

/gdrive/My Drive/lightDSFD/DSFDv2_r18


In [0]:
from humanfriendly import format_timespan
import time


In [59]:
from __future__ import print_function
import sys
import os
import argparse
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
from PIL import Image

from model_search import Network
from collections import OrderedDict

import numpy as np
import cv2
import math
import glob
import time
from dataset.config import widerface_640 as cfg
from torchvision.ops import nms as torch_nms

from collections import namedtuple

FPN_Genotype = namedtuple("FPN_Genotype", "Inter_Layer Out_Layer")
AutoFPN = FPN_Genotype(
    Inter_Layer=[
        [("sep_conv_3x3", 1), ("conv_1x1", 0)],
        [("sep_conv_3x3", 2), ("sep_conv_3x3", 0), ("conv_1x1", 1)],
        [("sep_conv_3x3", 3), ("sep_conv_3x3", 1), ("conv_1x1", 2)],
        [("sep_conv_3x3", 4), ("sep_conv_3x3", 2), ("conv_1x1", 3)],
        [("sep_conv_3x3", 5), ("sep_conv_3x3", 3), ("conv_1x1", 4)],
        [("sep_conv_3x3", 4), ("conv_1x1", 5)],
    ],
    Out_Layer=[],
)


thresh=0.3
path="/gdrive/My Drive/lightDSFD/DSFDv2_r18/videoFrames"
trained_model="/gdrive/My Drive/lightDSFD/DSFDv2_r18/dsfdv2_r18.pth"

def test_base_transform(image, mean):
    
    x = image.astype(np.float32)
    x -= mean
    x = x.astype(np.float32)
    return x


class TestBaseTransform:
    def __init__(self, mean):
        #self.size = size
        self.mean = np.array(mean, dtype=np.float32)

    def __call__(self, image, boxes=None, labels=None):
        return test_base_transform(image, self.mean), boxes, labels


cuda = True
if cuda and torch.cuda.is_available():
    torch.set_default_tensor_type("torch.cuda.FloatTensor")
else:
    torch.set_default_tensor_type("torch.FloatTensor")


net = Network(
    C=64,
    criterion=None,
    num_classes=2,
    layers=1,
    phase="test",
    search=False,
    searched_fpn_genotype=AutoFPN,
    searched_cpm_genotype=None,
    fpn_layers=1,
    cpm_layers=1,
    auxiliary_loss=False,
)

state_dict = torch.load(trained_model, map_location=lambda storage, loc: storage)
print("Pretrained model loading OK...")
new_state_dict = OrderedDict()
for k, v in state_dict.items():
    if "auxiliary" not in k:
        name = k[7:]  # remove module.
        new_state_dict[name] = v
    else:
        print("Auxiliary loss is used when retraining.")

net.load_state_dict(new_state_dict)
net.cuda()
net.eval()
print("Finished loading model!")

transform = TestBaseTransform((104, 117, 123))

def preprocess(img):
    x = torch.from_numpy(transform(img)[0]).permute(2, 0, 1)
    x = x.unsqueeze(0).cuda()
    return x
    

save_path ="/gdrive/My Drive/res18_boxes" 
# os.makedirs(save_path, exist_ok=True)

files = glob.glob(os.path.join(path, "*.jpg"))

with torch.no_grad():
    for f in files:
        begin_time = time.time()
        img_name = f
        img_cv2 = cv2.imread(f)

        # pre-processing
        img = preprocess(img_cv2)
        
        # post-processing
        detections = net(img).view(-1, 5)
        # scale each detection back up to the image
        scale = torch.Tensor(
            [
                img_cv2.shape[1],
                img_cv2.shape[0],
                img_cv2.shape[1],
                img_cv2.shape[0]
            ]
        ).cuda()
        scores = detections[..., 0]
        boxes = detections[..., 1:] * scale
        
        # filter the boxes whose score is smaller than 0.8
        keep_mask = (scores >= thresh) & (boxes[..., -1] > 2.0)
        scores = scores[keep_mask]
        boxes = boxes[keep_mask]
        # print(scores.max())

        keep_idx = torch_nms(boxes, scores, iou_threshold=0.4)
        if len(keep_idx) > 0:
            keep_boxes = boxes[keep_idx].cpu().numpy()
            keep_scores = scores[keep_idx].cpu().numpy()

            for box, s in zip(keep_boxes, keep_scores):
                box = np.array(box, np.int32)
                cv2.rectangle(img_cv2, (box[0], box[1]), (box[2], box[3]), color=(0, 0, 255), thickness=2)
                cv2.rectangle(img_cv2, (box[0], box[1]-20), (box[0]+80, box[1]-2), color=(0, 255, 0), thickness=-1) 
                cv2.putText(img_cv2, "{:.2f}".format(s), (box[0], box[1]-2), cv2.FONT_HERSHEY_SIMPLEX, thresh, color=(255,255,255), thickness=2)
        a=img_name[-30:]
        # print(a)
        cv2.imwrite('/gdrive/My Drive/lightDSFD/DSFDv2_r18/res18_boxes'+a, img_cv2)
        print('/gdrive/My Drive/lightDSFD/DSFDv2_r18/res18_boxes'+a)
        end_time = time.time() - begin_time
        print("Total time for a frame is : ", format_timespan(end_time))
        print("{} faces are detected in {}.".format(len(keep_idx), img_name))

Streaming output truncated to the last 5000 lines.
0 faces are detected in /gdrive/My Drive/lightDSFD/DSFDv2_r18/videoFrames/28M43S_1583812723.mp4_604.jpg.
/gdrive/My Drive/lightDSFD/DSFDv2_r18/res18_boxes/28M43S_1583812723.mp4_605.jpg
Total time for a frame is :  0.16 seconds
0 faces are detected in /gdrive/My Drive/lightDSFD/DSFDv2_r18/videoFrames/28M43S_1583812723.mp4_605.jpg.
/gdrive/My Drive/lightDSFD/DSFDv2_r18/res18_boxes/28M43S_1583812723.mp4_606.jpg
Total time for a frame is :  0.18 seconds
0 faces are detected in /gdrive/My Drive/lightDSFD/DSFDv2_r18/videoFrames/28M43S_1583812723.mp4_606.jpg.
/gdrive/My Drive/lightDSFD/DSFDv2_r18/res18_boxes/28M43S_1583812723.mp4_607.jpg
Total time for a frame is :  0.15 seconds
0 faces are detected in /gdrive/My Drive/lightDSFD/DSFDv2_r18/videoFrames/28M43S_1583812723.mp4_607.jpg.
/gdrive/My Drive/lightDSFD/DSFDv2_r18/res18_boxes/28M43S_1583812723.mp4_608.jpg
Total time for a frame is :  0.16 seconds
0 faces are detected in /gdrive/My Drive/

In [60]:
print("Done da")

Done da
